### Experimenting with BERTScore

In [1]:
import bert_score

In [2]:
o = bert_score.score(["I am Marius"], ["My name is marius"], lang="en")

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Experimenting with Roberta tokenizer

In [17]:
from transformers import RobertaTokenizer

In [18]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
test = tokenizer("I am Marius")

In [19]:
out = tokenizer("I am Marius")

In [20]:
out

{'input_ids': [0, 100, 524, 1127, 6125, 2], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [21]:
test.input_ids

[0, 100, 524, 1127, 6125, 2]